In [6]:
import datetime
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt 
plt.style.use('seaborn')
%matplotlib qt

In [7]:
df = pd.read_csv('covid.csv')
df = df.drop('SNo', axis=1)
df

,ObservationDate,Province/State,Country/Region,Last Update,Confirmed,Deaths,Recovered
0,01/22/2020,Anhui,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
1,01/22/2020,Beijing,Mainland China,1/22/2020 17:00,14.0,0.0,0.0
2,01/22/2020,Chongqing,Mainland China,1/22/2020 17:00,6.0,0.0,0.0
3,01/22/2020,Fujian,Mainland China,1/22/2020 17:00,1.0,0.0,0.0
4,01/22/2020,Gansu,Mainland China,1/22/2020 17:00,0.0,0.0,0.0
...,...,...,...,...,...,...,...
9730,03/28/2020,Wyoming,US,2020-03-28 23:11:06,82.0,0.0,0.0
9731,03/28/2020,Xinjiang,Mainland China,2020-03-28 23:11:06,76.0,3.0,73.0
9732,03/28/2020,Yukon,Canada,2020-03-28 23:11:06,4.0,0.0,0.0
9733,03/28/2020,Yunnan,Mainland China,2020-03-28 23:11:06,180.0,2.0,172.0


In [8]:
def get_country(country_name):
    data = df[df['Country/Region']==country_name]
    data = data[data['Confirmed']>=100]
    country = data.groupby('ObservationDate')
    country = country.sum()
    return country

In [13]:
mexico = get_country('Mexico')['Confirmed']/129.2e6*1e6
days = mexico.size
print(days)
plt.plot(np.arange(1, days+1), mexico, linewidth=4, label='Mexico')

countries = ['US','France', 'Spain', 'Germany', 'Italy','South Korea','Mainland China']
population = [327.2e6, 66.99e6, 46.66e6, 82.79e6, 60.48e6, 51.47e6, 1400e6]
for i in range(0,len(countries)):
    country = get_country(countries[i])['Confirmed']/population[i]*1e6
    plt.plot(np.arange(1, 10+1), country[:10], linewidth=1.5, label=countries[i]) 

plt.ylabel('Casos por 1 millon de personas')
plt.xlabel('Días desde que el número de infectados es mayor a 100')
plt.legend()
plt.savefig('comparision.png', dpi=400)
plt.show()

10


In [14]:
mexico = pd.read_csv('mexico.csv')
mexico = mexico.drop('Caso', axis=1)
#mexico

In [15]:
states = mexico.groupby('Estado')
infected = states['Status'].count()
infected = infected.rename({'CIUDAD DE MÉXICO ': 'DISTRITO FEDERAL', 'MÉXICO ': 'MEXICO', 'NUEVO LEÓN ':'NUEVO LEON', 'SAN LUIS POTOSÍ ':'SAN LUIS POTOSI'})
infected = infected.sort_index()
infected.to_csv('infected.csv')
#infected

In [23]:
infected

EDO_LEY
Aguascalientes          19
Baja California         20
Baja California Sur      9
Campeche                 3
Chiapas                  6
Chihuahua                6
Coahuila                15
Colima                   2
Distrito Federal       177
Durango                  7
Guanajuato              37
Guerrero                10
Hidalgo                 12
Jalisco                 82
Mexico                  85
Michoacan               17
Morelos                  4
Nayarit                  5
Nuevo Leon              64
Oaxaca                  12
Puebla                  52
Queretaro               25
Quintana Roo            33
San Luis Potosi         20
Sinaloa                 14
Sonora                  10
Tabasco                 27
Tamaulipas               6
Tlaxcala                 4
Veracruz                23
Yucatan                 38
Zacatecas                4
Name: Status, dtype: int64

In [16]:
plt.hist(mexico['Edad'], bins=20, range=(0,100), color='royalblue', histtype='bar', ec='black')
plt.xlabel('Edad')
plt.ylabel('Número de infectados')
plt.show()
plt.savefig('histograma.png', dpi=400)

In [17]:
path = 'PHLITL_2000.shp'
data = gpd.read_file(path)

In [18]:
counties = data[['EDO_LEY', 'geometry']]
states = counties.dissolve(by='EDO_LEY')
states = states.sort_index()

In [19]:
infected.index=states.index
states['infected'] = infected
#states

In [22]:
plt.style.use('seaborn')
vmin = infected.min(); vmax = infected.max();
states.plot(column='infected', cmap='Blues', norm=plt.Normalize(vmin=vmin,vmax=vmax), linewidth=0.3, edgecolor='.8')

sm = plt.cm.ScalarMappable(cmap='Blues', norm=plt.Normalize(vmin=vmin, vmax=vmax))
sm._A = []
plt.colorbar(sm)

plt.title('Casos confirmados COVID-19', fontdict={'fontsize': '18', 'fontweight' : '3'})
plt.annotate('Fuente: Secretaría de Salud',xy=(0.15, .2),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=10, color='#555555')
plt.axis('off')
plt.show()

plt.savefig('mapa.png', dpi=400)